# Flask - продолжение

## Словари в шаблонах

Предположим, у нас есть словарь, в котором хранятся имена наших друзей и их почтовые адреса. И мы хотим выводить эти имена с адресами на нашем сайте. Мы можем написать что-то такое:

In [ ]:
from flask import Flask
from flask import render_template

app = Flask(__name__)


@app.route('/')
def index():
    emailbook = {'Петя': 'petya@example.com',
                 'Вася': 'vasya@example.com',
                 'Катя': 'katya@example.com'}
    return render_template('index.html', emails=emailbook)

if __name__ == '__main__':
    app.run()

Тогда в папке `templates` нам нужно создать файл `index.html`, в котором будут перебираться элементы словаря. Делается это с помощью функции `items()`. Вот так будет выглядеть `index.html`:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Почтовые адреса</title>
</head>
<body>
<h1>Адреса моих друзей</h1>
<ul>
    {% for name, email in emails.items() %}
        <li>{{ name }} - {{ email }}</li>
    {% endfor %}
</ul>
</body>
</html>

## Перенаправление

Для того, чтобы направить пользователя на другую страницу, используется функция `redirect`. Например, вот приложение, в котором есть страница /time. С помощью функции `redirect` можно, например, реализовать такое:

- если пользователь заходит на страницу /time в рабочее время (с 10 до 18), то он перенаправляется на главную страницу сайта,
- если пользователь заходит на страницу в другое время, то он перенаправляется на страницу /hi.

In [ ]:
import datetime

from flask import Flask
from flask import url_for, render_template, request, redirect

app = Flask(__name__)


@app.route('/')
def index():
    return '<html><body><h1>Привет, мир!</h1></body></html>'


@app.route('/hi')
@app.route('/hi/<user>')
def hi(user=None):
    if user is None:
        user = 'friend'
    return '<html><body><p>Привет, ' + user + '!</p></body></html>'


@app.route('/time')
def time_redirect():
    h = datetime.datetime.today().hour
    if 10 < h < 18:
        return redirect(url_for('index'))
    return redirect(url_for('hi'))


if __name__ == '__main__':
    app.run(debug=True)

# Русская морфология в питоне

Есть два модуля для работы с русским языком - pymorphy2  и pymystem3. Сегодня мы разберем pymystem3.

## Pymystem3

Устанавливаем pymystem3:

`pip install pymystem3`

И проверим, работает ли:

In [8]:
from pymystem3 import Mystem  # импортируем майстем
m = Mystem()  # создаем экземпляр класса-анализатора

У класса Mystem есть два метода:
* lemmatize, возвращающий список лемм,
* analyze, возвращающий полный морфологический разбор в виде словаря.

In [9]:
text = """
Как там ваш блог?
Да, неплохо, весьма неплохо.
Вы ведь и слова там не написали?
Вы только что написали:
"все еще осталась проблема доверия".
"""

Пример с `lemmatize`:

In [10]:
lemmas = m.lemmatize(text)
print(''.join(lemmas))


как там ваш блог?
да, неплохо, весьма неплохо.
вы ведь и слово там не написать?
вы только что написать:
"все еще оставаться проблема доверие".



Пример с `analyze`:

In [11]:
ana = m.analyze(text)
print(ana)

[{'text': '\n'}, {'text': 'Как', 'analysis': [{'gr': 'ADVPRO=', 'lex': 'как'}]}, {'text': ' '}, {'text': 'там', 'analysis': [{'gr': 'ADVPRO=', 'lex': 'там'}]}, {'text': ' '}, {'text': 'ваш', 'analysis': [{'gr': 'APRO=(вин,ед,муж,неод|им,ед,муж)', 'lex': 'ваш'}]}, {'text': ' '}, {'text': 'блог', 'analysis': [{'gr': 'S,муж,неод=(вин,ед|им,ед)', 'lex': 'блог'}]}, {'text': '?'}, {'text': '\n'}, {'text': 'Да', 'analysis': [{'gr': 'CONJ=', 'lex': 'да'}]}, {'text': ', '}, {'text': 'неплохо', 'analysis': [{'gr': 'ADV=', 'lex': 'неплохо'}]}, {'text': ', '}, {'text': 'весьма', 'analysis': [{'gr': 'ADV=', 'lex': 'весьма'}]}, {'text': ' '}, {'text': 'неплохо', 'analysis': [{'gr': 'ADV=', 'lex': 'неплохо'}]}, {'text': '.'}, {'text': '\n'}, {'text': 'Вы', 'analysis': [{'gr': 'SPRO,мн,2-л=им', 'lex': 'вы'}]}, {'text': ' '}, {'text': 'ведь', 'analysis': [{'gr': 'PART=', 'lex': 'ведь'}]}, {'text': ' '}, {'text': 'и', 'analysis': [{'gr': 'CONJ=', 'lex': 'и'}]}, {'text': ' '}, {'text': 'слова', 'analysis

## Пример сайта

Предположим мы пишем сайт, в котором есть форма для текста. Пользователь вставляет туда текст, отправляет форму, и получает в ответ тот же текст, в котором каждому слову приписана его часть речи. 

Вот код приложения на фласке, у меня он находится в файле `my_app.py`.

In [ ]:
from flask import Flask
from flask import url_for, render_template, request, redirect
from pymystem3 import Mystem

m = Mystem()
app = Flask(__name__)


def add_POS(text):
    result = ''
    ana = m.analyze(text)
    for i in ana:
        result += i['text']
        if i['text'].strip() and 'analysis' in i and i['analysis']:
            pos = i['analysis'][0]['gr'].split('=')[0].split(',')[0]
            result += '<span class="pos">{}</span>'.format(pos)
    return result


@app.route('/', methods=['get', 'post'])
def index():
    if request.form:
        text = request.form['text']
        result = add_POS(text).replace('\n', '<br>')
        return render_template('index_page.html', input=text, text=result)
    return render_template('index_page.html')


if __name__ == '__main__':
    app.run(debug=True)

А вот код шаблона `index_page.html`, который лежит  в папке `templates`:

# Выкладываем

Мы с вами будем выкладывать сайты на [www.pythonanywhere.com/](https://www.pythonanywhere.com/user/2017learnpython/consoles/) - ресурс, где можно бесплатно хостить небольшие приложения. Выкладывать туда приложения очень просто:

1. Нужно зайти на сайт и зарегистрироваться. 

    - Обратите внимание, что ваш юзернейм будет адресом вашего сайта. Например, если у вас юзернейм noname, то сайт будет доступен по адресу noname.pythonanywhere.com. 
    
    - После регистрации нужно будет подтвердить свой почтовый адрес.
    
2. Когда вы заходите на сайт, вы автоматически попадаете в Консоли (__Consoles__). Чтобы выложить приложение на фласке, нужно перейти в Веб (__Web__) - или пройти по ссылке `https://www.pythonanywhere.com/user/ВАШ_ЮЗЕРНЕЙМ/webapps/` - и нажать кнопку __`Add a new web app`__

    - Вам сообщат, что адрес сайта будет связан с вашим юзернеймом. Кликаем Next.
    - Теперь нужно выбрать фреймворк, на котором написан сайт. Кликаем Flask.
    - Теперь нужно выбрать версию питона. Кликаем на ту, которой вы пользовались, когда писали приложение. Например, Python 3.5.
    - Нужно ввести путь к файлу с приложением на сервере. Вам автоматически предлагают что-то вроде `/home/ВАШ_ЮЗЕРНЕЙМ/mysite/flask_app.py`, но я, например, поменяю на  `/home/ЮЗЕРНЕЙМ/mysite/my_app.py`, потому что мой код приложения на фласке находится в файле my_app.py.
    
3. Теперь нужно загрузить наши файлы на сервер.  
    
    - Переходим во вкладку Файлы (__Files__), там в списке директорий слева (Directories) находим наш сайт (__mysite__) и переходим туда. 
    - В эту папку нужно загрузить наш файл `my_app.py` и папку `templates`. Интерфейс там довольно понятный: новую папку можно создать слева, загрузить файл - справа.
    - (На самом деле, файлы можно не грузить руками. Вместо этого, можно сначала выложить весь ваш сайт на гитхаб, а потом зайти в Консоли (__Consoles__), и оттуда склонировать ваш реп с сайтом на сервер pythonanywhere в папку `mysite` командой `git clone ССЫЛКА_С_ГИТХАБА mysite`)
    
4. Теперь все готово, кроме одного - на сервере нет майстема. Для этого нам нужно создать виртуальное окружение

    - Переходим в Консоли (__Consoles__) и нажимаем __Bash__. Пишем там команды:
    - `mkvirtualenv myvirtualenv --python=/usr/bin/python3.5`  - это создаст виртуальное окружение, а именно папку myvirtualenv, в которой будет установлен ваш собственный новый отдельный питон3.5.
    - `pip install pymystem3` - это установит в ваше виртуальное окружение майстем
    - `pip install flask` - это установит в ваше виртуальное окружение фласк
    - `which python` - эта команда распечатает путь к вашему питону. Например, `/home/ВАШ_ЮЗЕРНЕЙМ/.virtualenvs/myvirtualenv/bin/python`. Из этого пути нужно убрать `/bin/python`, скопировать полученное, вернуться во вкладку Веб (__Web__) и вставить наш путь в секции `Virtualenv`. Нужно вставить что-то такое - `/home/ВАШ_ЮЗЕРНЕЙМ/.virtualenvs/myvirtualenv`.
    - Теперь нужно перезапустить приложение - кликаем большую зеленую кнопку Reload наверху. 

Готово! Мы можем перейти по ссылке - например, http://2017learnpython.pythonanywhere.com/ - и пользоваться нашим приложением!
    



    
    


# Домашнее задание

1) Написать аналогичный сайт с формой, в которую пользователь вставляет текст. Ваш сайт должен выводить всю информацию о глаголах в данном тексте: 

   * Вывести, сколько всего глаголов в тексте и какова доля глаголов в тексте,
   * Вывести, сколько переходных, сколько непереходных глаголов, сколько глаголов совершенного и несовершенного вида,
   * Вывести список всех глагольных лемм в порядке убывания их частотности.
    
2) Зарегистрироваться на pythonanywhere и выложить туда ваш код. Убедитесь, что сайт заработал. Вставьте ссылку на ваш сайт в [гугл-форму](https://goo.gl/forms/BQg5cZbtP4KJlNs83). Этот сайт мы будем постепенно доделывать, и он станет вашим экзаменационным проектом.

3) Кроме того, код вашего сайта нужно выложить на гитхаб.

___________
    
Возможно, у вас есть какая-то идея интересного приложения на фласке с использованием mystem (например, вы хотите не про глаголы, а про существительные). В таком случае, напишите мне письмо с описанием идеи, и если идея подойдет, то вы сможете реализовать ее.